# Подготовка среды

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import shutil
# shutil.copytree("/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT/embeddings/DeepPavlov-polit_3/bin", "/content/data/bin2")

'/content/data/bin2'

In [2]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [3]:
folder = "bert-base-multilingual-cased-polit3"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))
if not os.path.isdir(os.path.join("embeddings", folder, "tests")):
    os.mkdir(os.path.join("embeddings", folder, "tests"))

In [4]:
!pip install transformers
!pip install razdel

     |████████████████████████████████| 2.3MB 6.6MB/s 
     |████████████████████████████████| 3.3MB 25.9MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 


In [5]:
import torch
import nltk
import re
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

In [6]:
def my_dump(file, name):
    with open("./" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./" + name + ".pic", 'rb') as f:
        return pc.load(f)

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [ ]:
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

#text processing

## generate (split) sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "bin/all_cyberleninka_polit2"
sent1 = my_load(name)

CPU times: user 9.48 s, sys: 2.82 s, total: 12.3 s
Wall time: 15.5 s


In [ ]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

2348859

##process

In [ ]:
# text = "Тюмень стала самым «счастливым» городом России по уровню довольных своим городом людей."

def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings2(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)

    token_vecs_sum = list(map(sum_tensors, token_embeddings))

    new_tokens = []
    new_token_vecs = []
    j = -1
    for i in range(len(tokenized_text)):
        if len(tokenized_text[i]) > 2 and tokenized_text[i][0] == "#" and tokenized_text[i][1] == "#":
            new_tokens[j] = new_tokens[j] + tokenized_text[i][2:]
            new_token_vecs[j].append(token_vecs_sum[i])
        else:
            new_tokens.append(tokenized_text[i])
            new_token_vecs.append([token_vecs_sum[i]])
            j+=1
    new_token_vecs2 = []
    for i in new_token_vecs:
        new_token_vecs2.append(sum(i)/len(i))
    return new_tokens, new_token_vecs2

In [ ]:
def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)
    
    token_vecs_sum = list(map(sum_tensors, token_embeddings))
    return tokenized_text, token_vecs_sum

# token_vecs = hidden_states[-2][0]
# sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
# %%time
batch = 1000
start_time = time.time()

ftom_b = 633
to_b = 667

print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

for j in range(ftom_b, to_b):
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        tt, tvs = get_embeddings2(sent[i])
        for i in range(len(tt)):
            if tt[i].isalpha():# and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)
    # res.update(pr_tr)

    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# for i in pr_tr:
#     print(i, len(pr_tr[i]))

senteces from 633000 to 667000
done 00633 --- time: 139.61818027496338	--- total: 139.61859512329102
done 00634 --- time: 145.62771081924438	--- total: 285.26813793182373
done 00635 --- time: 161.94093108177185	--- total: 447.24002623558044
done 00636 --- time: 154.72393703460693	--- total: 602.0114665031433
done 00637 --- time: 152.00363969802856	--- total: 754.0553164482117
done 00638 --- time: 145.88600850105286	--- total: 899.9791722297668
done 00639 --- time: 153.29189777374268	--- total: 1053.295251607895
done 00640 --- time: 150.26451992988586	--- total: 1203.58762550354
done 00641 --- time: 150.73610496520996	--- total: 1354.3576035499573
done 00642 --- time: 149.62504148483276	--- total: 1504.0163288116455
done 00643 --- time: 143.0403561592102	--- total: 1647.0857734680176
done 00644 --- time: 130.23590421676636	--- total: 1777.3523845672607
done 00645 --- time: 139.0942177772522	--- total: 1916.4733715057373
done 00646 --- time: 142.24152874946594	--- total: 2058.74659037590

In [ ]:
r1 = {}
for i in res:
    r1[i] = res[i][0]

In [ ]:
# get_word_m(r1["войной"], r1, 10)

{'война': 0.7984782457351685,
 'войной': 1.0,
 'войны': 0.866584062576294,
 'второй': 0.8478848934173584,
 'гитлеровской': 0.8020665645599365,
 'гражданской': 0.7895058989524841,
 'ермании': 0.7855548858642578,
 'мировой': 0.8973607420921326,
 'последовавший': 0.7955130338668823,
 'фашизмом': 0.7791929244995117}

#aggregation of embeddings



In [ ]:
# folder = "bert-base-multilingual-cased-liter"

##list of files

In [ ]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

##get count

In [ ]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 5min 36s, sys: 12.6 s, total: 5min 49s
Wall time: 9min 39s


In [ ]:
my_dump(from_files, "embeddings/{}/from_files".format(folder))
my_dump(all_count, "embeddings/{}/all_count".format(folder))

## aggregate

In [ ]:
from_files = my_load("embeddings/{}/from_files".format(folder))
all_count = my_load("embeddings/{}/all_count".format(folder))

In [ ]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
out[:20]

[('в', 635344),
 ('и', 542639),
 ('с', 212433),
 ('на', 183593),
 ('не', 138957),
 ('что', 111833),
 ('к', 102092),
 ('как', 95143),
 ('а', 89003),
 ('по', 84289),
 ('о', 65967),
 ('для', 62878),
 ('из', 53393),
 ('г', 52932),
 ('это', 48842),
 ('от', 47410),
 ('его', 44910),
 ('м', 43412),
 ('но', 42292),
 ('the', 40495)]

In [ ]:
keys = list(all_count.keys())
k1 = []
k2 = []
k3 = []
kb = []
l = 0
for i in keys:
    l = len(i)
    if l == 1:
        k1.append(i)
    elif l == 2:
        k2.append(i)
    elif l == 3:
        k3.append(i)
    else:
        kb.append(i)
len(k1),len(k2),len(k3),len(kb)

(94, 1655, 9061, 481857)

In [ ]:
all_words = []
for i in kb:
    if all_count[i] > 10:
        all_words.append(i)
len(all_words)

73442

In [ ]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        # with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        with open("/content/data/bin2/{}".format( name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [ ]:
# batch = 250
# st_time = time.time()

# ftom_b = 0
# to_b = 4

# print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

# all_emb = {}
# for b in range(ftom_b, to_b):
#     t_time = time.time()
#     words = all_words[b*batch:(b+1)*batch]
#     emb = get_all_embeddings(words)
#     res_emb = sum_embeddings(emb)
    
#     with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
#         pc.dump(res_emb, f)

#     print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

In [ ]:
def get_batch_number(max_i, from_i = 0):
    
    files_r = os.listdir("./embeddings/{}/ready".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1]))
    diff = set(range(from_i, max_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(b))
    my_dump("file", "./embeddings/{}/ready/{}_{:05d}_empty".format(folder,batch, b))

batch = 1000
from_i = 0
st_time = time.time()
print(" max batchs = {}".format( len(all_words)//batch))

all_emb = {}
l = get_batch_number(len(all_words)//batch + 1, from_i)
while len(l) > 0:
    b = l[0]
    reserve_batch(b)
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))
    l = get_batch_number(len(all_words)//batch + 1, from_i)

 max batchs = 73
reserve 0
done: 0 --- time: 1343.4875259399414	--- total: 1343.4998941421509
reserve 2
done: 2 --- time: 1378.8886213302612	--- total: 2722.421571969986
reserve 4
done: 4 --- time: 1424.6429018974304	--- total: 4147.082001447678
reserve 6
done: 6 --- time: 1464.5511395931244	--- total: 5611.662657260895
reserve 7
done: 7 --- time: 1442.6339089870453	--- total: 7054.317224740982
reserve 5
done: 5 --- time: 1419.7109065055847	--- total: 8474.049374580383
reserve 9
done: 9 --- time: 1431.0483038425446	--- total: 9905.11210179329
reserve 11
done: 11 --- time: 1359.502784729004	--- total: 11264.637256860733
reserve 13
done: 13 --- time: 1358.453075170517	--- total: 12623.110010147095
reserve 15
done: 15 --- time: 1392.8894081115723	--- total: 14016.021415948868
reserve 17
done: 17 --- time: 1294.0197718143463	--- total: 15310.061240673065
reserve 19
done: 19 --- time: 1292.7326505184174	--- total: 16602.81778693199
reserve 21
done: 21 --- time: 1256.4940950870514	--- total:

## delete trash

In [ ]:
files = os.listdir("./embeddings/{}/ready".format(folder))
trash_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        if len(i.split('.')[0].split("_")) == 3:
            trash_files.append(i)

In [ ]:
len(trash_files)

39

In [ ]:
for file in trash_files:
    os.remove("./embeddings/{}/ready/{}".format(folder,file))
    print("./embeddings/{}/ready/{}\t removed!".format(folder,file))

#building dictionary


In [ ]:
# folder = "bert-base-multilingual-cased-liter"

In [ ]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [ ]:
# with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
#     json.dump(lista_items, f, indent=4)

NameError: ignored

#Tests

##preparation

In [7]:
# folder = "bert-base-multilingual-cased-liter"
# folder = "bert-base-multilingual-cased-liter2"
# folder = "bert-base-multilingual-cased-liter3"
# folder = "bert-base-multilingual-cased-polit"
# folder = "bert-base-multilingual-cased-polit2"
# folder = "bert-base-multilingual-cased-polit3"
# folder = "DeepPavlov-liter"
# folder = "DeepPavlov-liter_2"
# folder = "DeepPavlov-liter_3"
# folder = "DeepPavlov-polit"
# folder = "DeepPavlov-polit_2"
folder = "DeepPavlov-polit_3"

In [8]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "br") as f:
    dct1  = pc.load(f)

# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
#     dct1  = pc.load(f)

In [9]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])

def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [10]:
def save_table(df, name):
    df.to_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

def get_table( name):
    return pd.read_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

In [ ]:
# list(dct1.keys())

In [11]:
# get_word_m(dct1["россия"], dct1, 10)

##GATS

###generate test

In [12]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [13]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [14]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [15]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [16]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [17]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "bw") as f:
    pc.dump(words_rus, f)

##get quality


In [18]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "br") as f:
    words_rus = pc.load( f)

In [19]:
def cut_b(text):
    # return tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")[1]
    return text

###3cosadd

In [ ]:
def cos_add(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
            # print(m[2], pred)
    return res_df

In [ ]:
n = 3
def cos_add_3(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred = get_word_m(dct[m[0]] - dct[m[1]] + dct[m[3]], dct,count = n)
            if m[2] in pred:
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], m[2], pred[m[2]], key]
            else:
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], list(pred.keys())[0], pred[list(pred.keys())[0]], key]
            # print(m[2], pred)
    return res_df

In [ ]:
for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_add(key, dct1)
    save_table(res_df1, "3cosadd_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [ ]:
groups = [": capital-world",
": family",
": gram1-adjective-to-adverb",
": gram2-opposite",
": gram3-comparative",
": gram6-nationality-adjective"]

res_df1 = pd.DataFrame()
for key in groups:
    res_df1 = pd.concat([res_df1, get_table("3cosadd_res_{}".format(key))], ignore_index = True )

In [ ]:
# get_table("3cosadd_res_{}".format(key))
res_df1

,Unnamed: 0,w1,w2,w3,expected,prdicted,cosine,group
0,0,алжир,алжир,иордания,амман,иордании,0.964762,: capital-world
1,1,алжир,алжир,турция,анкара,турции,0.963184,: capital-world
2,2,алжир,алжир,туркменистан,ашхабад,туркменистана,0.976596,: capital-world
3,3,алжир,алжир,казахстан,астана,казахстана,0.974255,: capital-world
4,4,алжир,алжир,греция,афины,греции,0.966985,: capital-world
...,...,...,...,...,...,...,...,...
2206,580,украина,украинский,голландский,нидерланды,голландской,0.907172,: gram6-nationality-adjective
2207,581,украина,украинский,норвежский,норвегия,норвежские,0.941645,: gram6-nationality-adjective
2208,582,украина,украинский,польский,польша,украину,0.928379,: gram6-nationality-adjective
2209,583,украина,украинский,испанский,испания,украину,0.924537,: gram6-nationality-adjective


In [ ]:
# res_df1_3 = cos_add_3(dct1)
# save_table(res_df1_3, "3cosadd_3_res")

for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_add_3(key, dct1)
    save_table(res_df1, "3cosadd_3_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [ ]:
# res_df1["group"].unique()

res_df1 = pd.DataFrame()
for key in groups:
    res_df1 = pd.concat([res_df1, get_table("3cosadd_res_{}".format(key))], ignore_index = True )

In [ ]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Nationality adjective,0.837703,0.374359,585


In [ ]:
save_table(res_sort_1, "3cosadd_itog")

In [ ]:
# res_df1["group"].unique()

res_df1_3 = pd.DataFrame()
for key in groups:
    res_df1_3 = pd.concat([res_df1_3, get_table("3cosadd_3_res_{}".format(key))], ignore_index = True )

In [ ]:
res_df1_3["result"] = res_df1_3["expected"]  == res_df1_3["prdicted"] 
count_1 = res_df1_3.groupby("group").count()
res_3_sort_1 = res_df1_3.groupby("group").mean().sort_values(by = "result")
res_3_sort_1["count"] = count_1["w1"]
res_3_sort_1 = add_names(res_3_sort_1)
res_3_sort_1.index = res_3_sort_1["type"]
res_3_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.711428,0.000000,20
Capital-countries,0.833677,0.030093,864
Family,0.712448,0.250000,20
Nationality adjective,0.837703,0.374359,585
Adjective adverb,0.922219,0.402632,380
Comparative,0.866328,0.479532,342


In [ ]:
save_table(res_3_sort_1, "3cosadd_3_itog")

In [ ]:
# cut_b("камбоджа")

###3cosavg

In [20]:
def cos_avg_2(key, dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
                m.append(cut_b(i))

    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            res_df.loc[res_df.shape[0]] = [dct[m[0]].numpy(), 
                                            dct[m[1]].numpy(), 
                                            dct[m[3]].numpy(), 
                                            dct[m[2]].numpy(),
                                            key]
    dif = res_df["w1"].mean() - res_df["w2"].mean()
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])

    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
        # if m[2] in dct and m[3] in dct:
            pred, prob = get_word(dif + dct[m[3]].numpy(), dct, nin = [m[0], m[1], m[3]])
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
            # print(m[2], pred)
    return res_df
for key in words_rus:
    print("start {}".format(key))
    res_df1_avg = cos_avg_2(key, dct1)
    save_table(res_df1_avg, "3cosavg_2_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [21]:
res_df1_avg = pd.DataFrame()
for key in words_rus.keys():
    res_df1_avg = pd.concat([res_df1_avg, get_table("3cosavg_2_res_{}".format(key))], ignore_index = True )

res_df1_avg["result"] = res_df1_avg["expected"]  == res_df1_avg["prdicted"] 
count_1 = res_df1_avg.groupby("group").count()
res_df1_avg2 = res_df1_avg.groupby("group").mean().sort_values(by = "result")
res_df1_avg2["count"] = count_1["w1"]
res_df1_avg2 = add_names(res_df1_avg2)
res_df1_avg2.index = res_df1_avg2["type"]
save_table(res_df1_avg2, "3cosavg_res_2")
res_df1_avg2[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.861808,0.000000,56
Capital-countries,0.926686,0.023573,806
Adjective adverb,0.934652,0.230769,650
Comparative,0.897142,0.423280,756
Nationality adjective,0.945351,0.720137,586
Family,0.951152,0.727273,110


In [ ]:
def cos_avg(dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    for key in words_rus:
        res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                
                    m.append(cut_b(i))
                # else:
                #     m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                res_df.loc[res_df.shape[0]] = [dct[m[0]].numpy(), 
                                               dct[m[1]].numpy(), 
                                               dct[m[3]].numpy(), 
                                               dct[m[2]].numpy(),
                                               key]
                # print(m[2], pred)
            
            
        w1,p1 = get_word(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), dct)
        w2,p2 = get_word(res_df["expected"].mean(), dct)
        prob = 1 - cosine(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), res_df["expected"].mean())
        res.loc[res.shape[0]] = [w1,p1,w2,p2, prob, key, res_df.shape[0]]
    return res

In [ ]:
res_df1_avg = cos_avg(dct1)
res_df1_avg.index = res_df1_avg["key"]
add_names(res_df1_avg)
res_df1_avg.index = res_df1_avg["type"]
res_df1_avg[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,рабат,0.922577,рабат,0.922810,0.998820,864
Family,братья,0.929720,братья,0.929719,1.000000,20
Adjective adverb,ощутимый,0.947901,ощутимый,0.947901,1.000000,380
Opposite,приемлемый,0.952114,приемлемый,0.952114,1.000000,20
Comparative,жесткий,0.952476,жесткий,0.952476,1.000000,342
Nationality adjective,германия,0.940812,германия,0.940813,0.999997,585


In [ ]:
save_table(res_df1_avg, "3cosavg_res")

###3cosmul


In [ ]:
def get_word_mul(w1,w2,w3, dct):
    word = "none"
    cs = -100 
    for key in dct:
        e1 = 1 - cosine(dct[w1], dct[key])
        e2 = 1 - cosine(dct[w2], dct[key])
        e3 = 1 - cosine(dct[w3], dct[key])
        if e1 - e2 + e3 > cs and key != w1 and key != w2 and key != w3:
            cs = e1 - e2 + e3
            word = key
    return word, cs

In [ ]:
def cos_mul(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            
            m.append(cut_b(i))
            # else:
            #     m.append("none")
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word_mul(m[0], m[1], m[3], dct)
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [ ]:
for key in [": gram6-nationality-adjective"]:
    print("start {}".format(key))
    res_df1 = cos_mul(key, dct1)
    save_table(res_df1, "3cosmul_res_{}".format(key))
    print("finish {}".format(key))

start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [ ]:
groups = [": capital-world",
": family", 
": gram1-adjective-to-adverb",
": gram2-opposite",
": gram3-comparative",
": gram6-nationality-adjective"]

res_df1_mul = pd.DataFrame()
for key in groups:
    res_df1_mul = pd.concat([res_df1_mul, get_table("3cosmul_res_{}".format(key))], ignore_index = True )

In [ ]:
# res_df1_mul = cos_mul(dct1)

# save_table(res_df1_mul, "3cosmul_res")
res_df1_mul["result"] = res_df1_mul["expected"]  == res_df1_mul["prdicted"] 
count_1 = res_df1_mul.groupby("group").count()
res_df_1_mul = res_df1_mul.groupby("group").mean().sort_values(by = "result")
res_df_1_mul["count"] = count_1["w1"]
res_df_1_mul = add_names(res_df_1_mul)
res_df_1_mul.index = res_df_1_mul["type"]
res_df_1_mul[["cosine", "result", "count"]]
save_table(res_df_1_mul, "3cosmul_itog")

In [ ]:
save_table(res_df1_mul, "3cosmul_res")
save_table(res_df_1_mul, "3cosmul_itog")

###mix table

In [ ]:
ddd1 = pd.DataFrame()
def get_r1(r1, metric):
    r1["metric"] = metric
    return r1.pivot(columns = ["metric"])

In [ ]:
ddd1 = get_r1(res_sort_1[["cosine", "result", "count"]], "3CosAdd")
ddd1 = ddd1.merge(get_r1(res_df_1_mul[["cosine", "result", "count"]], "3CosMul"), on = "type")
r1 = res_df1_avg[["prob", "count"]].copy()
r1["cosine"] = r1["prob"]
ddd1 = ddd1.merge(get_r1(r1[["cosine", "count"]], "3CosAvg"), on = "type")
ddd1

In [ ]:
ddd1[["result", "cosine", "count"]]